SOLID is an acronym where each letter represents some principles of software development. Let's go through them.

"S" stands for Single Responsibility Principle. It states that classes/functions should have one job. So if a class has more than one responsibility it becomes "coupled", meaning a change to one responsibility results in the modification of the other responsibility. Let's highlight an instance where this is not done well.

In [ ]:
class User:
  def __init__(self, name: str):
    self.name = name

  def get_name(self) -> str:
    pass

  def get_user_from_db(self, id) -> User:
    pass

  def save_to_db(self, user: User):
    pass

We have a User class which is responsible for both the user properties and user database management. If the database management functions have to change, then the class that makes use of the User properties will have to be changed as well. The solution would be to decouple this into two classes.

In [ ]:
class User:
  def __init__(self, name: str):
    self.name = name
  
  def get_name(self):
    pass


class UserDB:
  def get_user(self, id) -> User:
    pass

  def save(self, user: User):
    pass

Some additional thinking on this can be found in the [Facade pattern](https://kennison.name/files/zopestore/uploads/python/DesignPatternsInPython_ver0.1.pdf).

"O", or the "Open-Closed" principle, states that classes/functions should be open to extension, but closed to modification. When we add new features to our codebase, we prefer to define new objects or extend existing objects, not change existing objects that are closed to modification. Let's start with a sub-optimal example.

In [ ]:
class Employee:
  def __init__(self, name: str, salary: str):
    self.name = name
    self.salary = salary
  
class Recruiter(Employee):
  def __init__(self, name: str, salary: str):
    super().__init__(name, salary)
    
  def recruit(self):
      print("{} is testing".format(self.name))

class Developer(Employee):
  def __init__(self, name: str, salary: str):
    super().__init__(name, salary)
  
  def develop(self):
    print("{} is developing".format(self.name))

class Company:    
  def __init__(self, name: str):
    self.name = name
  
  def work(self, employee):
    if isinstance(employee, Developer):
      employee.develop()
    elif isinstance(employee, Recruiter):
      employee.recruit()
    else:
      raise Exception("Unknown employee")

This code has shortcomings as it pertains to defining a new kind of Employee. The "Developer" class has a method called `develop()`, and the "Recruiter" class has a method called `recruit()`. If we chose to add a "Salesman", we would have to modify the Company class `work()` method. We can instead define an abstract class (ABC class) with an abstract method called `work()` that would require its child classes to implement a method of that name. Then, we can use that method in the Company class regardless of which type of Employee is doing the work.

In [ ]:
from abc import ABC, abstractmethod

class Employee(ABC):
  def __init__(self, name: str, salary: str):
    self.name = name
    self.salary = salary

  @abstractmethod
  def work(self):
    pass

class Recruiter(Employee):
  def __init__(self, name: str, salary: str):
    super().__init__(name, salary)

  def work(self):
    print("{} is recruiting".format(self.name))

class Developer(Employee):
  def __init__(self, name: str, salary: str):
    super().__init__(name, salary)

  def work(self):
    print("{} is developing".format(self.name))

class Company:
  def __init__(self, name: str):
    self.name = name

  def do_work(self, employee: Employee):
    employee.work()

pin = Company("PIN")
developer = Developer("Tapia", "1000000")
recruiter = Recruiter("Shane", "1000000")
pin.do_work(developer)
pin.do_work(recruiter)

The next principle is called the "Liskov’s Substitution" principle, or the "L" principle. The Liskov Substitution Principle states that objects in a program should be replaceable with instances of their subtypes without altering the correctness of that program.

In other words, if a class 'Square' is a subtype of a class 'Rectangle', then in the program a 'Square' object should be able to substitute a 'Rectangle' object without needing to change the program. Here's an example that breaks that rule.

In [4]:
class Rectangle:
  def __init__(self, width=0, height=0):
    self.width = width
    self.height = height

  def area(self):
    return self.width * self.height

class Square(Rectangle):
  def __init__(self, side):
    # A Square is always a Rectangle, but a rectangle is not always a square.
    # If we wanted our Square to substitue a Rectangle, it should be able to
    # implement the area() method. For now, the area() method is broken.
    super().__init__()
    self.side = side

rect = Rectangle(2, 3)
print(rect.area()) # 6

sq = Square(2)
print(sq.area()) # Should be 4

6
0


Here is an example that satisfies the rule. We now pass in the sides of the Square as the width and height, thus allowing the area() method to run as expected. Now, we can substitute the subtype "Square" for the supertype "Rectangle" and the program will still work.

In [6]:
class Rectangle:
  def __init__(self, width=0, height=0):
    self.width = width
    self.height = height

  def area(self):
    return self.width * self.height

class Square(Rectangle):
  def __init__(self, side):
    super().__init__(side, side)

rect = Rectangle(2, 3)
print(rect.area()) # 6

sq = Square(2)
print(sq.area()) # 4

6
4


Let's look at different example that breaks this principle. Say we have a parent class Car that would store some details about a vehicle. The Car class is inherited by subclass PetrolCar. Similarly, the parent class Car can be inherited by other classes which would extend the features as needed for each implementation of the child class.

In [ ]:
class Car():
	def __init__(self, type):
		self.type = type

class PetrolCar(Car):
	def __init__(self, type):
		self.type = type

car = Car("SUV")
car.properties = {"Color": "Red", "Gear": "Auto", "Capacity": 6}
print(car.properties)

petrol_car = PetrolCar("Sedan")
petrol_car.properties = ("Blue", "Manual", 4)
print(petrol_car.properties)

Here there is no method defined to add properties of the Car and custom code is required to implement properties. The very type used to implement the properties (ex. dict vs tuple above) is left to the developer to choose. Let’s say that there is a requirement to find all red cars with a function and see if both the superclass (parent class of "Car") and subclass (child class of "PetrolCar") work with the given function.

In [ ]:
class Car():
	def __init__(self, car_type):
		self.car_type = car_type

class PetrolCar(Car):
	def __init__(self, car_type):
		self.car_type = car_type

car = Car("SUV")
car.properties = {"Color": "Red", "Gear": "Auto", "Capacity": 6}
print(car.properties)

petrol_car = PetrolCar("Sedan")
petrol_car.properties = ("Blue", "Manual", 4)
print(petrol_car.properties)

cars = [car, petrol_car]

def find_red_cars(cars):
	red_cars = 0
	for car in cars:
		if car.properties['Color'] == "Red":
			red_cars += 1
	print(f"Number of Red cars: {red_cars}")

find_red_cars(cars)

As we can see here, we are trying to loop through a list of car objects. But we break the Liskov Substitution principle as we cannot replace super class objects with sub class objects in the function. A simple solution would be to introduce setter and getter methods in the super class Car, thus telling the sub class how to get/set properties and not leaving the implementation details to the developer. We can even add typings to the "car_properties" attribute to ensure that it is a Dictionary with keys that are strings and values that are of any type. Do remember that using typings doesn't throw an error if it is not followed, but rather that a type checker can be run on your code and that it will inform you of incorrect typing.

Now the code is friendly towards Liskov’s Substitution Principle.

In [7]:
from typing import Any, Dict

class Car():
	def __init__(self, car_type):
		self.car_type = car_type
		self.car_properties: Dict[str, Any] = {}
	
	def set_properties(self, color, gear, capacity):
		self.car_properties = {"Color": color, "Gear": gear, "Capacity": capacity}
	
	def get_properties(self):
		return self.car_properties

class PetrolCar(Car):
	def __init__(self, car_type):
		self.car_type = car_type

car = Car("SUV")
car.set_properties("Red", "Auto", 6)

petrol_car = PetrolCar("Sedan")
petrol_car.set_properties("Blue", "Manual", 4)

cars = [car, petrol_car]

def find_red_cars(cars):
	red_cars = 0
	for car in cars:
		if car.get_properties()["Color"] == "Red":
			red_cars += 1
	print(f"Number of Red cars: {red_cars}")

find_red_cars(cars)

Number of Red cars: 1


Onto "I". The Interface Segregation Principle (ISP) states that no client (or 'class') should be forced to implement interfaces (or 'methods') it does not use.

This is an example that does not satisfy ISP. It requires the child classes to define methods it does not need. Using the @abstractmethod, any class that inherits an ABC (or 'Abstract Base Class') must implement these methods or it will throw an error. Therefore, PdfDocument and WordDocument would have to define all three methods of `save()`, `print()`, and `email()`. This code block will throw an error like "TypeError: Can't instantiate abstract class PdfDocument with abstract methods email, print".

In [8]:
from abc import ABC, abstractmethod

class Document(ABC):
  @abstractmethod
  def save(self):
    pass

  @abstractmethod
  def print(self):
    pass

  @abstractmethod
  def email(self):
    pass

class PdfDocument(Document):
  def __init__(self):
    super().__init__() # Won't do anything until you want shared logic on subclasses from AbstractBase

  def save(self):
    print("Saving PDF document")

class WordDocument(Document):
  def __init__(self):
    super().__init__() # Won't do anything until you want shared logic on subclasses from AbstractBase

  def save(self):
    print("Saving Word document")

  def email(self):
    print("Emailing Word document")

pdf = PdfDocument()
pdf.save() # Saving PDF document
pdf.email() # Saving PDF document

word = WordDocument()
word.save() # Saving Word document
word.email() # Emailing Word document


TypeError: Can't instantiate abstract class PdfDocument with abstract methods email, print

Let's fix this by not making the subclasses implement every method from their parent. Now, the subclasses will only have to implement `save()`.

In [ ]:
from abc import ABC, abstractmethod

class Document(ABC):
  
  @abstractmethod
  def save(self):
    pass

  def print(self):
    pass

  def email(self):
    pass

class PdfDocument(Document):
  def __init__(self):
    super().__init__() # Won't do anything until you want shared logic on subclasses from AbstractBase

  def save(self):
    print("Saving PDF document")

class WordDocument(Document):
  def __init__(self):
    super().__init__() # Won't do anything until you want shared logic on subclasses from AbstractBase

  def save(self):
    print("Saving Word document")

  def email(self):
    print("Emailing Word document")

pdf = PdfDocument()
pdf.save() # Saving PDF document
pdf.email() # Saving PDF document

word = WordDocument()
word.save() # Saving Word document
word.email() # Emailing Word document

Here is another example of code that doesn't follow ISP. You can see that it is trying to define an object that would parse data from different formats - XML and JSON.

In [ ]:
from abc import ABC, abstractmethod

class EventParser(ABC):
	
	@abstractmethod
	def from_xml(xml_data: str):
		"Parse an event from XML"
	
	@abstractmethod
	def from_json(json_data: str):
		"Parse an event from JSON"

Why isn't this code correct? Well, what if an object that uses this interface doesn’t **need** to use the from_json method? Here, we force this class to implement an functional interface (which in this case is a funcion) they don’t want to use. The better approach is to create different class interfaces to support different functions, and therefore use-cases.

In [ ]:
from abc import ABC, abstractmethod

class XMLEventParser(ABC):
	
	@abstractmethod
	def from_xml(xml_data: str):
		"Parse an event from XML"
	
class JSONEventParser(ABC):
	
	@abstractmethod
	def from_json(json_data: str):
		"Parse an event from JSON"

# RestClients often consume JSON events
class RestClient(JSONEventParser):
	pass

# SoapClients often consume XML events
class SoapClient(XMLEventParser):
	pass

Last but certainly not least: Dependency Inversion, or "DIP".

The Dependency Inversion Principle states that:
a) High level module should not depend on low level modules. Both should depend on abstractions. (recall that a "module" could be a function, a class, a file, really any unit of logic)
b) Abstractions should not depend on details. Details should depend on abstractions.

In other words, the DIP suggests that higher-level components of a system should not be tightly coupled with low-level components. Instead, both high-level and low-level components should depend on abstractions.

Note: If your code follows the Open-Closed Principle and Liskov Substitution Principle, then it will be implicitly aligned to be compliant to the Dependency Inversion Principle also.

Imagine that you have a program that is given an input with characteristics that are consistent, like a .csv file with predefined columns, and you wrote a script to process it. What would happen if that file type or column structure were subject to change?

You'd have to rewrite your script and accomodate the new format! This would also mean that the older file type + structure wouldn't be compatible with the updated program.

However, you could instead solve this by creating another abstraction that takes the file as an input and passes it to the other interfaces. Let's start with a very simple example using this principle.

In [ ]:
"""
Let's say you have a program to bake bread.
In a higher level module/abstraction you'd likely call cook() somewhere.
A poor implementation would cook AND intialize the bread/bake it.
"""
def cook():
    bread = Bread()
    bread.bake()

cook()

As you can see, the cook function depends on the Bread object. So what happens if you want to bake cookies?

A novice mistake is to add a condition like this:

In [ ]:
def cook(food: str):
  if food == "bread":
    bread = Bread()
    bread.bake()
  if food == "cookies":
    cookies = Cookies()
    cookies.bake()

cook("cookies")

This code isn't great from a dependency perspective. By adding more food, you would have to change the cook interface and it will become a blob with many conditions - plus it breaks almost every principle!

You need the cook function (which is a higher level module) not to depend on lower level modules, like the Bread or Cookies objects. So, the only input we need is something that we can bake, then bake it. Now the right way to do this is by implementing an interface.

Let's invert this dependency!

In [ ]:
from abc import ABC, abstractmethod

class Bakeable(ABC):
	@abstractmethod
	def bake(self):
			pass

class Bread(Bakeable):
  def bake(self):
    print('Making bread.')
	
class Cookies(Bakeable):
  def bake(self):
    print('Yum, cookies.')

def cook(bakeable: Bakeable):
	bakeable.bake()

cookies = Cookies()
bread = Bread()

# Some client would inject a Bakeable object into the cook function like such. It can be a separate entity entirely,
# but for simplicity we just call the cook function here.
cook(cookies)
cook(bread)

And now the cook function depends on the **abstraction**. It doesn't depend on messy conditions, the Bread, or the Cookies, but on the Bakeable abstraction. Now any Bakeable can be baked using the cook function.

By implementing the interface we are sure that every Bakeable will have a bake() method that does something, as is required by children of an Abstract Class. Additionally, the cook function does not need to know which sub class is implemented! The cook function will simply bake anything that is Bakeable.

The dependency now goes to the client, which would be some logic that'd call the cook function. The client is the one that wishes to bake something, would import the cook function, observe that it wants a Bakeable, is the only part of this program that is concerned with having a Bakeable defined, and would ultimately pass it into the cook function.